Instructions
Faça o webscrapping do site:
https://pokemondb.net/pokedex/national
usando beaultifulsoup. Guarde as informações de todas as gerações de pokemon, inclusive com o link das imagens em um dataframe pandas e depois exporte como
CSV.

. importing libraries

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import base64
from IPython.display import display, Image

2. Dowloading da url

In [2]:
# URL do site
url = 'https://pokemondb.net/pokedex/national'


3.Faz uma requisição HTTP para obter o conteúdo da página:


3. O código abaixo faz o seguinte:

Faz uma requisição HTTP para obter o conteúdo da página.

Usa BeautifulSoup para analisar o HTML da página.

Encontra a tabela com o id 'pokedex' e, se encontrada, obtém todas as linhas (<tr>) dentro do corpo da tabela (<tbody>).

Se a tabela não for encontrada, define pokemon_rows como uma lista vazia.

In [3]:
# Faz uma requisição HTTP para obter o conteúdo da página.
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')


4. Este código faz o seguinte:

Faz uma requisição HTTP para obter o conteúdo da página.

Usa BeautifulSoup para analisar o HTML da página.

Encontra a tabela com o id 'pokedex' e, se encontrada, obtém todas as linhas 
(<tr>) dentro do corpo da tabela (<tbody>).

Se a tabela não for encontrada, define pokemon_rows como uma lista vazia.

In [4]:
# Faz uma requisição HTTP para obter o conteúdo da página.
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Usa BeautifulSoup para analisar o HTML da página.
# Encontra a tabela com o id 'pokedex' e, se encontrada, obtém todas as linhas (tr) dentro do corpo da tabela (tbody).
# Se a tabela não for encontrada, define pokemon_rows como uma lista vazia.
table = soup.find('table', {'id': 'pokedex'})
if table:
    pokemon_rows = table.find('tbody').find_all("tr")
else:
    pokemon_rows = []

5.  verify the content of pokemon_rows

In [5]:
# pokemon_rows is already defined in cell 10, no need to redefine it
print(pokemon_rows)  # To verify the content of pokemon_rows

[]


6. Listas para armazenar os dados:


In [6]:
generations = []
names = []
list_types = []
hps = []
attacks = []
defenses = []
sp_Atks = []
sp_Defs = []
speeds = []
full_powers = []
images = []

7. Criando a pasta para salvar as imagens:

In [7]:
if not os.path.exists('images'):
    os.makedirs('images')

8. Extraindo as informações de cada Pokémon, incluindo geração, nome, tipos, HP, ataque, defesa, ataque especial, defesa especial, velocidade, total de poder e imagem:


In [8]:
for pokemon in tqdm(pokemon_rows):
    if not pokemon.find("small") is None:
        generations.append(pokemon.find("small", class_='text-muted').text.strip())
    else:
        generations.append('')

    name = pokemon.find('a', class_='ent-name').text
    names.append(name)

    aux = pokemon.find_all("a", class_='itype')
    qtd_power = len(aux)
    types = ''
    for power in range(qtd_power):
        types += aux[power].text
        if power < qtd_power - 1 and qtd_power > 1:
            types += ' e '
    list_types.append(types)

    full_power = int(pokemon.find("td", class_='cell-num cell-total').text)
    full_powers.append(full_power)

 
    image_url = pokemon.find("img", class_='img-fixed icon-pkmn')["src"]

     # Verificação do URL da imagem
    print(f"Baixando imagem de: {image_url}")

    # Salva as imagens localmente na pasta images.
    image_response = requests.get(image_url)
    image_path = os.path.join('images', f'{name}.png')
    with open(image_path, 'wb') as f:
        f.write(image_response.content)

    # Codifica a imagem em base64
    with open(image_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
    images.append(encoded_image)

    skill = pokemon.find_all("td", class_='cell-num')[2:]
    hps.append(int(skill[0].text))
    attacks.append(int(skill[1].text))
    defenses.append(int(skill[2].text))
    sp_Atks.append(int(skill[3].text))
    sp_Defs.append(int(skill[4].text))
    speeds.append(int(skill[5].text))

0it [00:00, ?it/s]


9. Armazena as informações em listas:

In [9]:
df = pd.DataFrame({
    'Generation': generations,
    'Name': names,
    'Type': list_types,
    'HP': hps,
    'Attack': attacks,
    'Defense': defenses,
    'Sp. Atk': sp_Atks,
    'Sp. Def': sp_Defs,
    'Speed': speeds,
    'Total Power': full_powers,
    'Image': images
})

9. Cria um DataFrame do pandas com as informações:

In [10]:
df = pd.DataFrame({
    'Generation': generations,
    'Name': names,
    'Type': list_types,
    'HP': hps,
    'Attack': attacks,
    'Defense': defenses,
    'Sp. Atk': sp_Atks,
    'Sp. Def': sp_Defs,
    'Speed': speeds,
    'Total Power': full_powers,
    'Image': images
})

10. Exporta o DataFrame para um arquivo CSV chamado pokemon_data.csv:

In [11]:
# Exporta o DataFrame para um arquivo CSV chamado pokemon_data.csv.
df.to_csv('pokemon_data.csv', index=False)

print('Dados exportados para pokemon_data.csv e imagens salvas na pasta images')

Dados exportados para pokemon_data.csv e imagens salvas na pasta images


11. Exibir as imagens no notebook:

In [12]:
for image_path in images:
    display(Image(filename=image_path))